# Prompt Templates and Dynamic Prompts

PromptTemplates in Langchain enables dynamic insertion of variables into standardized prompt structures.

A PromptTemplate is a blueprint for creating dynamic prompts. Instead of writing static
prompts, you create templates with placeholders (variables) that get filled in at runtime.

## Bootstrap

⚓--- Before proceeding futher it is very important you do the following: --- 👾

Select the 🗝 (key) icon in the left pane and include your OpenAI Api key with Name as "OPENAPI_KEY" and value as the key, and grant it notebook access in order to be able to run this notebook.

Run the below two cells in the order they are in, before running further cells. Wait till a number appears in place of '*' or '[ ]'. Below the cell you should see "Ready. LangChain + OpenAI set up."

In [ ]:
!pip install -q langchain langchain-openai langchain-community pydantic pypdf faiss-cpu

In [ ]:
# Bootstrap: environment & model (same as earlier)
from google.colab import userdata

key = userdata.get('OPENAI_API_KEY')  # returns None if not granted
if not key:
    raise RuntimeError("Set OPENAI_API_KEY in a .env file next to this notebook.")

from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, FewShotPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2, streaming=True, api_key=key)

print("✅ Ready: Prompt Templates with LCEL")

✅ Ready: Prompt Templates with LCEL


## PromptTemplate

The most basic prompt template in Langchain is `PromptTemplate`. You can use `PromptTemplates` of any kind in one of two ways. Using `.format` to fill the variable value or directly in a chain using `.invoke` by placing the template variable value in a dictionary with the variable name as the key.

In [ ]:
# Define a single-message text template
summary_tpl = PromptTemplate.from_template(
    "Summarize the following text in 2 sentences:\n\n{text}"
)

# Use it standalone
prompt_text = summary_tpl.format(text="LangChain lets you build LLM apps by composing prompt, model, and tools.")
print("Rendered Prompt:\n", prompt_text)

# Use it in a chain (Prompt → Model → Parser)
summary_chain = summary_tpl | llm | StrOutputParser()
result = summary_chain.invoke({"text": "LangChain lets you build LLM apps by composing prompt, model, and tools."})

print("\n--- Final Output ---")
print(result)

Rendered Prompt:
 Summarize the following text in 2 sentences:

LangChain lets you build LLM apps by composing prompt, model, and tools.

--- Final Output ---
LangChain enables the creation of large language model (LLM) applications by integrating prompts, models, and various tools. This compositional approach simplifies the development process for building sophisticated LLM-based solutions.


## ChatPromptTemplate

ChatPromptTemplate is specifically designed for chat-based interactions. It handles the conversation structure with system messages, human messages, and AI messages, making it perfect for ChatGPT-style applications.

### Messages

ChatPromptTemplate requires Messages as inputs. These Messages (also called Message roles) are used to define the speaker of each message in a conversation. For user the message is `HumanMessage`, for the LLM response `AIMessage`, for system message (more on this later) it is `SystemMessage`. There is also `ToolMessage` which represents the message by the tool called (Tool-calling).

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a precise assistant that uses markdown bullets."),
    ("user", "Explain {topic} in 3 short bullets.")
])

chain = chat_prompt | llm | StrOutputParser()
print("\n--- Final Output ---")
print(chain.invoke({"topic": "Prompt Templates in LangChain"}))

## Chat history and Messages placeholder

ChatPromptTemplate accepts a list of messages as input making them perfect for Conversational chatbots with chat history.

For this purpose we'll be using `MessagesPlaceholder` which allow for dynamic injection of messages into a prompt.

### How it works

MessagesPlaceholder is used within a `ChatPromptTemplate.from_messages()` call. It takes a single argument: the variable name that will be used to pass in the list of messages.

Let's see how we can use `MessagesPlaceholder` for Chat history.

In [ ]:
# Define the chat prompt template with a placeholder for history
messages_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{user_input}")
])

# Let's create a chat history list
chat_history = [
    HumanMessage(content="What's the capital of France?"),
    AIMessage(content="The capital of France is Paris.")
]

messages_chain = messages_prompt | llm | StrOutputParser()

result = messages_chain.invoke({
    "chat_history": chat_history,
    "user_input": "What's the famous food in the city?"
})

print(result)

Now, without previous context for the prompt "What's the most famous food in the city?" you wouldn't be getting the response you'd expect.

## Few shot prompts

Few shot prompt is a famous prompt that shows examples of the desired instruction-response pairs for the AI to mimic. We can achieve this Langchain in many ways but Langchain has an inbuilt PromptTemplate for the same.

In [ ]:
sentiment_examples = [
    {
        "text": "I absolutely love this product! It exceeded all my expectations.",
        "sentiment": "Positive",
        "confidence": "High"
    },
    {
        "text": "The service was okay, nothing special but not terrible either.",
        "sentiment": "Neutral",
        "confidence": "Medium"
    },
    {
        "text": "Completely disappointed with this purchase. Total waste of money.",
        "sentiment": "Negative",
        "confidence": "High"
    }
]

example_template = PromptTemplate(
    input_variables=["text", "sentiment", "confidence"],
    template="Text: {text}\nSentiment: {sentiment}\nConfidence: {confidence}"
)

sentiment_few_shot = FewShotPromptTemplate(
    examples=sentiment_examples,
    example_prompt=example_template,
    prefix="Analyze the sentiment of the following texts. For each text, provide the sentiment (Positive/Negative/Neutral) and confidence level (High/Medium/Low).\n\nExamples:",
    suffix="\nText: {input_text}\n",
    input_variables=["input_text"]
)

sentiment_chain = sentiment_few_shot | llm | StrOutputParser()

print(sentiment_chain.invoke({"input_text":"The movie was quite entertaining, though the ending felt rushed."}))

### How it works

You need a few examples that are a list of dictionaries. You need to create a Prompt template based on the list of dictionaries. When the llm is invoked with this template, based on the template and the example list, if the list contains n dictionaries n prompts are created in the structure of the `PromptTemplate` and then added to the input prompt (`FewShotPromptTemplate`). User query and additional context can be added before or after this using prefix or suffix.

## Validation of Prompt templates

Langchain validates required variables at render time.

In [ ]:
try:
    bad = PromptTemplate.from_template("Hello {name}, you are {role}.")
    print(bad.format(name="Ada"))  # missing 'role'
except Exception as e:
    print("Validation error:", e)

Validation error: 'role'


This way you can validate whether or not a prompt is receiving the required values when building applications.